In [1]:
# Ý tưởng:
# Biểu diễn mỗi ký tự bằng one-hot
# Ở mỗi bước t:
#     Thực hiện tính công thức trạng thái ẩn
#     Đầu ra đưa qua softmax
#     loss là cross-entropy loss
# Backpropagation: Tính gradient từ T -> 1, cộng dồn qua thời gian
# Có gradient clipping tránh nổ gradient
import numpy as np

In [2]:
def one_hot(idx, vocab_size):
    # vector v có vocab_size hàng và 1 cột
    v = np.zeros((vocab_size, 1))
    # Cả hàng idx sẽ toàn giá trị 1.0
    v[idx] = 1.0
    return v
def softmax(z):
    exp = np.exp(z)
    return exp / np.sum(exp)


In [3]:
v = np.zeros((3, 2))
v[1] = 1.0
v

array([[0., 0.],
       [1., 1.],
       [0., 0.]])

In [ ]:
# RNN thuần
class RNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        # Khởi tạo trọng số gồm value + và -
        s = 0.01
        # ma trận gồm hidden_size hàng, input_size cột
        self.Wxh = np.random.randn(hidden_size, input_size) * s
        # ma trận gồm hidden_size hàng, hidden_size cột
        self.Whh = np.random.randn(hidden_size, hidden_size) * s
        self.bh = np.zeros((hidden_size, 1))
        # ma trận gồm output_size hàng, hidden_size cột
        self.Why = np.random.randn(output_size, hidden_size) * s
        self.by = np.zeros((output_size, 1))
    def params(self):
        return {
            "Wxh": self.Wxh, "Whh": self.Whh, "bh": self.bh,
            "Why": self.Why, "by": self.by
        }
    def forward(self, inputs, prev_hidden_state):
        # inputs: list chỉ số của ký tự [x1, x2,...]
        # prev_hidden_state : hidden state ban đầu
        # return cache(xs, hs, os, ps) và hidden last
        # xs, hs, os, ps chứa các xt, ht, ot, pt trong công thức của RNN
        xs, hs, os, ps = {}, {}, {}, {}
        # hidden state trước t = 0
        hs[-1] = np.copy(prev_hidden_state)
        loss = 0.0
        for t, ix in enumerate(inputs):
            xs[t] = one_hot(ix, self.input_size)
            hs[t] = np.tanh(self.Wxh @ xs[t] + self.Whh @ hs[t-1] + self.bh)
            os[t] = self.Why @ hs[t] + self.by
            ps[t] = softmax(os[t])
        cache = (xs, hs, os, ps)
        # trả về hidden state cuối cùng làm input cho batch tiếp theo
        # trả về cache để tính backpropagation
        return cache, hs[len(inputs)-1]
    def loss_and_grads(self, inputs, targets, prev_hidden_state):
        (xs, hs, os, ps), hlast = self.forward(inputs, prev_hidden_state=prev_hidden_state)
        loss = 0.0
        # Tính loss
        for t in range(len(inputs)):
            loss += -np.log(ps[t][targets[t],0] + 1e-12)
        

In [5]:
wxh = np.random.randn(3, 5)
wxh

array([[ 1.34879863,  0.80673493,  2.01732609,  1.00962568, -0.71128231],
       [-0.37936053, -0.35164814, -0.54497114,  1.66028067, -0.21830353],
       [-0.12995255,  0.17360866,  0.39169758,  0.72847171, -0.27522642]])

In [6]:
s = 0.01
input_size = 4
hidden_size = 3
output_size = input_size
# ma trận gồm hidden_size hàng, input_size cột
Wxh = np.random.randn(hidden_size, input_size) * s
print("Wxh (hidden size, input size): \n",Wxh,"\n")
# ma trận gồm hidden_size hàng, hidden_size cột
Whh = np.random.randn(hidden_size, hidden_size) * s
print("Whh (hidden size, hidden size):\n",Whh, "\n")
bh = np.zeros((hidden_size, 1))
print("bh (hidden size):\n",bh, "\n")
# ma trận gồm output_size hàng, hidden_size cột
Why = np.random.randn(output_size, hidden_size) * s
print("Why (output size, hidden size): \n",Why, "\n")
by = np.zeros((output_size, 1))
print("by (output size): \n",by, "\n")

Wxh (hidden size, input size): 
 [[ 0.00501531 -0.00542057  0.0207437  -0.02535079]
 [ 0.006724    0.021044   -0.00072248  0.00302208]
 [ 0.00841048  0.01661836  0.00136355  0.000384  ]] 

Whh (hidden size, hidden size):
 [[ 0.0086436  -0.02502921 -0.00266082]
 [-0.03130735 -0.01074938 -0.00490338]
 [-0.00184732 -0.0058222  -0.00522432]] 

bh (hidden size):
 [[0.]
 [0.]
 [0.]] 

Why (output size, hidden size): 
 [[-0.00606778  0.00577124  0.01541739]
 [ 0.02415788 -0.01382654  0.00969458]
 [-0.00489479  0.0059564  -0.00358817]
 [-0.0096404   0.01133897  0.01170806]] 

by (output size): 
 [[0.]
 [0.]
 [0.]
 [0.]] 



In [ ]:
inputs = [1, 2, 3]
input_size = 4
xs, hs, os, ps = {}, {}, {}, {}
hs[-1] = np.zeros((hidden_size,1))
for t, ix in enumerate(inputs):
    xs[t] = one_hot(ix, input_size)
    hs[t] = np.tanh(Wxh @ xs[t] + Whh @ hs[t-1] + bh)
    os[t] = Why @ hs[t] + by
    ps[t] = softmax(os[t])
    print(f"Giá trị thứ {t} của input là:\n{xs[t]}\nCó hidden state thứ {t} là:\n{hs[t]}")
    print(f"Output thứ {t} của input thứ {t} là:\n {os[t]}\nCó xác suất:\n {ps[t]}")
    print("---------------------------------------------------------")

Giá trị thứ 0 của input là:
[[0.]
 [1.]
 [0.]
 [0.]]
Có hidden state thứ 0 là:
[[-0.00542051]
 [ 0.0210409 ]
 [ 0.01661683]]
Output thứ 0 của input thứ 0 là:
 [[ 4.10510691e-04]
 [-2.60777709e-04]
 [ 9.22361351e-05]
 [ 4.85388767e-04]]
Có xác suất:
 [[0.25005716]
 [0.24988936]
 [0.24997759]
 [0.25007589]]
---------------------------------------------------------
Giá trị thứ 1 của input là:
[[0.]
 [0.]
 [1.]
 [0.]]
Có hidden state thứ 1 là:
[[ 0.02012328]
 [-0.00086044]
 [ 0.00116425]]
Output thứ 1 của input thứ 1 là:
 [[-0.00010912]
 [ 0.00050932]
 [-0.0001078 ]
 [-0.00019012]]
Có xác suất:
 [[0.24996632]
 [0.25012096]
 [0.24996665]
 [0.24994607]]
---------------------------------------------------------
Giá trị thứ 2 của input là:
[[0.]
 [0.]
 [0.]
 [1.]]
Có hidden state thứ 2 là:
[[-0.0251531 ]
 [ 0.00239561]
 [ 0.00034575]]
Output thứ 2 của input thứ 2 là:
 [[ 0.00017178]
 [-0.00063742]
 [ 0.00013615]
 [ 0.0002737 ]]
Có xác suất:
 [[0.25004642]
 [0.24984416]
 [0.25003751]
 [0.250071